In [29]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn as sk
from sklearn import linear_model

import tensorflow as tf

In [6]:
#data starts from 05-01-2024. 

df = pd.read_excel('AAPL_Intraday_Data.xlsx')
df.head()



,Dates,Last Price,Bid Price,Ask Price,Bid-Ask Spread,Volume,Turnover
0,2024-05-01 09:30:00,170.000,169.94,170.05,0.11,704606.0,119685384.0
1,2024-05-01 09:31:00,169.770,169.98,170.05,0.07,194725.0,33083506.0
2,2024-05-01 09:32:00,169.620,169.98,170.07,0.09,110680.0,18798334.0
3,2024-05-01 09:33:00,169.350,169.98,170.07,0.09,203403.0,34494664.0
4,2024-05-01 09:34:00,169.115,170.00,169.80,-0.20,254573.0,43094548.0


In [7]:
#CHECK TO SEE IF THERE IS ANYTHING WRONG
#results show some NaT and NaN values
#should look into this when cleaning data
df.tail()

,Dates,Last Price,Bid Price,Ask Price,Bid-Ask Spread,Volume,Turnover
32090,NaT,NaN,NaN,207.27,207.27,NaN,NaN
32091,NaT,NaN,NaN,207.22,207.22,NaN,NaN
32092,NaT,NaN,NaN,207.22,207.22,NaN,NaN
32093,NaT,NaN,NaN,207.21,207.21,NaN,NaN
32094,NaT,NaN,NaN,207.30,207.30,NaN,NaN


In [8]:
#notice how the data type is datetime64 and float64
#all of them should be fine(?)
print(df.dtypes)

Dates             datetime64[ns]
Last Price               float64
Bid Price                float64
Ask Price                float64
Bid-Ask Spread           float64
Volume                   float64
Turnover                 float64
dtype: object


In [9]:
#change the name of the columns for easier access/read
df.columns = [
    'DATES',
    'PRICE',
    'BID',
    'ASK',
    'SPREAD',
    'VOLUME',
    'TURNOVER'
]

### Linear Regression
#### Y = what we are looking for. 
Looking for predicted price of AAPL for the next day

AAPL opened at $207.72 on 6/24/2024

In [10]:
#attempt with the implications of predicted stock price for each (run 7 models)

Y = df['PRICE']
X = df[['BID','ASK','SPREAD','VOLUME','TURNOVER']]

In [11]:
data = pd.concat([Y,X],axis=1)
data.describe()

,PRICE,BID,ASK,SPREAD,VOLUME,TURNOVER
count,14090.000000,31901.000000,32095.000000,32095.000000,1.409000e+04,1.409000e+04
mean,193.653786,193.765112,193.785265,1.191377,1.457984e+05,2.871314e+07
std,11.560946,11.441971,11.401349,16.165544,1.012050e+06,2.077106e+08
min,169.115000,169.100000,169.130000,-12.250000,0.000000e+00,0.000000e+00
25%,186.760000,186.720000,186.750000,-0.340000,4.763600e+04,9.034876e+06
50%,191.380000,191.260000,191.300000,0.030000,7.581550e+04,1.459399e+07
75%,196.010000,196.080000,196.030000,0.360000,1.310178e+05,2.558792e+07
max,220.058000,220.050000,220.080000,207.840000,1.066871e+08,2.213760e+10


In [12]:
#use mean of stocks to fill NaN values
# Fill NaN values with the mean of the column
data_clean = data.dropna(subset = ['PRICE'])

In [13]:
#this is showing me that I dropped more than half my data in the case where I drop NA values
#HOWEVER, there should be about 14000 data considering this is an accumulation of 36ish days of trading.
#making the dropna VALID!
data_clean.describe()

,PRICE,BID,ASK,SPREAD,VOLUME,TURNOVER
count,14090.000000,14090.000000,14090.000000,14090.000000,1.409000e+04,1.409000e+04
mean,193.653786,185.129803,185.162511,0.032708,1.457984e+05,2.871314e+07
std,11.560946,5.791221,5.777881,0.393016,1.012050e+06,2.077106e+08
min,169.115000,169.100000,169.130000,-2.620000,0.000000e+00,0.000000e+00
25%,186.760000,182.550000,182.600000,-0.110000,4.763600e+04,9.034876e+06
50%,191.380000,185.200000,185.140000,0.040000,7.581550e+04,1.459399e+07
75%,196.010000,189.940000,189.970000,0.170000,1.310178e+05,2.558792e+07
max,220.058000,192.750000,192.760000,6.180000,1.066871e+08,2.213760e+10


In [14]:
#reassign X and Y to the new columns with no NaNs

Y = data_clean['PRICE']
X = data_clean[['BID','ASK','SPREAD','VOLUME','TURNOVER']]

In [15]:
data_clean.tail()

,PRICE,BID,ASK,SPREAD,VOLUME,TURNOVER
14085,207.795,191.30,190.84,-0.46,1063631.0,2.210122e+08
14086,208.040,191.40,190.85,-0.55,1376674.0,2.859571e+08
14087,208.020,191.30,190.84,-0.46,1463390.0,3.046166e+08
14088,207.490,191.30,190.83,-0.47,106687072.0,2.213760e+10
14089,207.490,191.32,190.83,-0.49,0.0,0.000000e+00


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [17]:
from sklearn import linear_model
OLS = linear_model.LinearRegression()

In [18]:
OLS.fit(X_train, y_train)

LinearRegression()

In [19]:
print("OLS.intercept_:", OLS.intercept_)
print("OLS.coef_:", OLS.coef_)

OLS.intercept_: -102.08793713378373
OLS.coef_: [ 9.38933638e-01  6.58898218e-01 -2.80035419e-01 -1.99594308e-05
  9.82307771e-08]


In [20]:
#obtain dictionary mapping column name with corresponding coefficient
d = {X.columns[i]:OLS.coef_[i] for i in range(0, len(OLS.coef_))}

In [21]:
s = pd.Series(d)
s

BID         9.389336e-01
ASK         6.588982e-01
SPREAD     -2.800354e-01
VOLUME     -1.995943e-05
TURNOVER    9.823078e-08
dtype: float64

In [22]:
y_pred = OLS.predict(X_test)
y_pred.shape

(3523,)

In [28]:
#scores correspond to R^2. Discussed more below
print("Training set score: {:.4f}".format(OLS.score(X_train, y_train)))
print("Test set score: {:.4f}".format(OLS.score(X_test, y_test)))

Training set score: 0.6683
Test set score: 0.6740


#### From what I am understanding, the equation is as follows to predict AMZN stock price:
<ul>Assume closing price for all.
</ul>
    AAPL = (BID*0.9389336)+(ASK*0.6588982)+(SPREAD*-0.2800354)+(VOLUME*-0.00001995943)+(TURNOVER*0.00000009823078)-102.087937 
<ul>
    248.58 vs. 207.72 (6/24/2024 open)
    </ul>


### Overall, for how bad the training and test score was, the result isn't as bad as it could have been.